# 🤖 7-DOF Robot Topology Optimization using Pinocchio + CasADi

This notebook demonstrates how to optimize the link lengths of a 7-DOF robotic arm using CasADi and Pinocchio.  
The objective is to maximize the average manipulability over a set of sampled configurations.


In [6]:
import time
import numpy as np
import casadi
from casadi import SX, vertcat, Function, nlpsol
import pinocchio as pin
from pinocchio import casadi as cpin
import matplotlib.pyplot as plt
from pinocchio.visualize import MeshcatVisualizer
import meshcat
vis = meshcat.Visualizer()
vis.jupyter_cell()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/


## 🧩 1. Define link length variables
We define symbolic variables for the 7 link lengths and provide default initial values.


## 🏗️ 2. Build a 7-DOF planar arm in Pinocchio
Each joint is a revolute joint around the Y-axis (JointModelRY).
Links are aligned along X and their lengths are parameters.


In [7]:
def build_symbolic_robot(lengths_sx):
    model = cpin.Model()
    joint_placement = cpin.SE3.Identity()
    parent_id = 0

    for i in range(7):
        joint_name = f'joint{i}'
        joint_id = model.addJoint(parent_id, cpin.JointModelRY(), joint_placement, joint_name)
        inertia = cpin.Inertia.Random()
        model.appendBodyToJoint(joint_id, inertia, cpin.SE3.Identity())
        model.addFrame(cpin.Frame(f'link{i}', joint_id, 0, cpin.SE3.Identity(), cpin.FrameType.BODY))
        parent_id = joint_id

        joint_placement = cpin.SE3.Identity()
        joint_placement.translation = vertcat(lengths_sx[i], 0, 0)

    # model.addBodyFrame("ee", parent_id, cpin.SE3.Identity())
    return model



## 📐 3. Manipulability score function
This function computes the average Yoshikawa manipulability index over a set of random joint configurations.


In [8]:
def symbolic_manipulability(model, lengths_sx):
    nq = model.nq
    q = SX.sym("q", nq)

    data = cpin.Data(model)
    cpin.forwardKinematics(model, data, q)
    J = cpin.computeJointJacobian(model, data, q, model.njoints - 1)[:3, :]
    manipulability = casadi.sqrt(casadi.det(J @ J.T) + 1e-8)
    return Function("manip_score", [q], [manipulability])



## 🔧 4. Optimization with CasADi
We define an objective function that returns the negative manipulability score for given link lengths, and solve it using IPOPT.


In [9]:
def build_objective_function(lengths_sym):
    model = build_symbolic_robot(lengths_sym)
    manip_func = symbolic_manipulability(model, lengths_sym)
    nq = model.nq

    def objective_func(L_vals_np):
        manip_scores = []
        for _ in range(40):
            q_sample = np.random.uniform(-np.pi, np.pi, nq)
            manip_scores.append(manip_func(q_sample).full().item())
        return -np.mean(manip_scores)

    return objective_func




In [10]:
L_sym = SX.sym("L", 7)                     # 7 symbolic link lengths
objective_np = build_objective_function(L_sym)  # builds model, computes manip

obj_func = Function("f", [L_sym], [objective_np(L_sym)])  # wrap in CasADi Function

nlp = {"x": L_sym, "f": obj_func(L_sym)}
solver = nlpsol("solver", "ipopt", nlp)

L0 = np.array([0.4] * 7)
res = solver(x0=L0, lbx=0.1, ubx=1.0)
L_opt = res["x"].full().flatten()
print("✅ Optimal link lengths:", L_opt)



******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        7
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality c

## 🧑‍🎨 5. Visualize the optimized robot
We build the robot using the optimal link lengths and display it using Meshcat.


In [ ]:
# ⚠️ cette partie utilise un modèle NON-symbolique pour la visualisation (Meshcat ne gère pas SX)
from pinocchio import Model, Inertia, Frame, FrameType, SE3
import pinocchio as pin

def build_visual_model(lengths):
    model = Model()
    joint_placement = SE3.Identity()
    parent_id = 0

    for i in range(7):
        name = f'joint{i}'
        joint_id = model.addJoint(parent_id, pin.JointModelRY(), joint_placement, name)
        inertia = Inertia.Random()
        model.appendBodyToJoint(joint_id, inertia, SE3.Identity())
        model.addFrame(Frame(f'link{i}', joint_id, 0, SE3.Identity(), FrameType.BODY))
        parent_id = joint_id
        joint_placement = SE3(
            pin.utils.rpyToMatrix(0.0, 0.0, 0.0),
            np.array([lengths[i], 0.0, 0.0])
        )

    model.addBodyFrame("ee", parent_id, SE3.Identity())
    return model

# Visualisation
visual_model = build_visual_model(L_opt)
data = visual_model.createData()
viz = MeshcatVisualizer(visual_model, data, vis)
viz.initViewer()
viz.loadViewerModel()
viz.display(np.zeros(visual_model.nq))



ArgumentError: Python argument types in
    Model.addBodyFrame(Model, str, int, SE3)
did not match C++ signature:
    addBodyFrame(pinocchio::ModelTpl<double, 0, pinocchio::JointCollectionDefaultTpl> {lvalue} self, std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > body_name, unsigned long parentJoint, pinocchio::SE3Tpl<double, 0> body_placement, int previous_frame)

## ✅ Next steps

- Compute workspace volume using convex hull (e.g. Qhull)
- Add constraints: total arm length, center of mass position, collisions
- Include dynamic objectives like total inertia or torque-efficiency
- Use a real URDF model and extract symbolic parameters
